In [16]:
import os
import csv
import re
from fuzzywuzzy import fuzz
from pdfminer.high_level import extract_text
import codecs
import sys
from unidecode import unidecode
from PyPDF2 import PdfReader


# Ordner mit PDFs und CSV-Datei für Suchwörter angeben
pdf_folder = 'PDF/' #os.path.join(os.path.dirname(__file__), 'PDF/')
search_terms_file = 'search_words.csv' #os.path.join(os.path.dirname(__file__), 'search_words.csv')
output_file = 'results.csv' #os.path.join(os.path.dirname(__file__), 'results_fuzzy.csv')

Simple

In [20]:
def read_pdf(filename):
    with open(filename, 'rb') as f:
        pdf_text = extract_text(f, codec='utf-8')
    # Count the number of page breaks in the text to get the total number of pages
    page_count = pdf_text.count('\f')
    return pdf_text, page_count



with open(search_terms_file, 'r') as f:
    reader = csv.reader(f)
    search_words = [row[0] for row in reader]

with open(output_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['PDF', 'Page', 'Search Word', 'Context'])

for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith('.pdf'):
        pdf_text, total_pages = read_pdf(pdf_folder + pdf_file)
        for search_word in search_words:
            escaped_word = re.escape(search_word)
            # match_pattern = re.compile(search_word, re.IGNORECASE) # RegEx in CSV
            match_pattern = re.compile(escaped_word, re.IGNORECASE)
            for match in re.finditer(match_pattern, pdf_text):
                start_index = max(0, match.start() - 100)
                end_index = min(len(pdf_text), match.end() + 100)                
                context = pdf_text[start_index:end_index]
                context = re.sub(r'[^\w\s]+', '', context, flags=re.UNICODE).replace('\n', ' ')
                context = unidecode(context)
                context = context.replace('\f', '')
                context = re.sub(r'\s+', ' ', context)
                page_number = pdf_text[:match.start()].count('\f') + 1
                with open(output_file, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([pdf_file, page_number, search_word, context])

Fuzzy

In [36]:
# Funktion, die PDF-Seiten durchsucht und Übereinstimmungen mit Suchwörtern findet
def search_pdf_pages(pdf_file_path, search_terms):
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        page_count = len(pdf_reader.pages)
        matches = []
        
        for page_num in range(page_count):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text() if page.extract_text() else ""  # um sicherzustellen, dass page_text nicht None ist
            
            for term in search_terms:
                max_similarity = 0
                most_similar_word = ""

                for word in page_text.split():
                    if len(word) < 3:  # Filtern Sie kurze Wörter heraus
                        continue
                    current_similarity = fuzz.partial_ratio(word.lower(), term.lower())
                    if current_similarity > max_similarity:
                        max_similarity = current_similarity
                        most_similar_word = word

                if max_similarity >= 90:  # Oder ein anderer Schwellenwert nach Wahl
                    start_index = page_text.lower().find(most_similar_word.lower())
                    end_index = start_index + len(most_similar_word)

                    # Extrahieren Sie den Kontext um das gefundene Wort
                    context_start = max(0, start_index - 50)
                    context_end = min(len(page_text), end_index + 50)
                    context = page_text[context_start:context_end]

                    # Entfernen von Zeilenumbrüchen, Tabulatoren und doppelten Leerzeichen
                    context = re.sub(r'\s+', ' ', context).strip()

                    matches.append((pdf_file_path, page_num+1, term, max_similarity, most_similar_word, context))
                    
    return matches



# Liste mit Suchwörtern aus CSV-Datei erstellen
search_terms = []
with open(search_terms_file, 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        search_terms.extend(row)

# CSV-Datei für Ergebnisse öffnen und Spaltenüberschriften schreiben
result_file = open("fuzzy_" + output_file, 'w', newline='', encoding='utf-8')
csvwriter = csv.writer(result_file)
csvwriter.writerow(['PDF-Datei', 'Seite', 'Suchbegriff', 'Ähnlichkeit', 'Suchwort im Text'])

# Durch alle PDF-Dateien im Ordner iterieren und Suchfunktion aufrufen
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_folder, filename)
        matches = search_pdf_pages(file_path, search_terms)
        for match in matches:
            csvwriter.writerow([match[0], match[1], match[2], match[3], match[4], match[5]])

            
# CSV-Datei schließen
result_file.close()

print('Suche abgeschlossen. Ergebnisse sind in CSV gespeichert.')

Suche abgeschlossen. Ergebnisse sind in CSV gespeichert.


Topic